# Pong

Run this notebook and see what happens. 

Click the game canvas once it shows up to bring it into focus and then use the arrows and keys A and D to control the pedals.


In [1]:
import logging
import sys
import os
import math

import numpy as np

In [2]:
sys.path.insert(0, os.path.abspath('./..'))

In [3]:
import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

c:\Users\sgtas\AppData\Local\Programs\Python\Python39\lib\site-packages\pyglet\libs\win32\__init__.py:326: UserWarning: Could not set COM MTA mode. Unexpected behavior may occur.
  warnings.warn("Could not set COM MTA mode. Unexpected behavior may occur.")


In [4]:
app = App()#log_level=logging.INFO)

In [5]:
background = '#3e32a2'
foreground = '#7c71da'

In [6]:
body_im = np.ones((20, 120)) * 255
wheel_im = np.ones((15,40)) * 255
a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255

In [7]:
body = Sprite(body_im, y=app.height/2, x=app.width/2)
f_wheel = Sprite(wheel_im, y=app.height/2, x=app.width/2+60, color=foreground)
r_wheel = Sprite(wheel_im, y=app.height/2, x=app.width/2-60, color=foreground)
field = Sprite(a2, y=app.height/2, x=app.width/2, color=background)

In [10]:
speed = Label(
    '0', font_size=22, color=foreground, 
    x=100, y=20,
    anchor_y='center', anchor_x='left'
)

steering_angle = Label(
    '0', font_size=22, color=foreground, 
    x=100, y=44, 
    anchor_y='center', anchor_x='left'
)

In [11]:
@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)
    field.draw()
    body.draw()
    f_wheel.draw()
    r_wheel.draw()
    speed.draw()
    steering_angle.draw()

In [12]:
state = State( 
    x = app.width / 2,
    y = app.height / 2,
    
    body_angle = 0.0,
    steering_angle = 0.0,
    
    l = 50.0,
    speed = 0.0,
    
    turn_left = False,
    turn_right = False,

    speed_up = False,
    slow_down = False,
)

In [13]:
@app.event
def key_event(key, action, modifiers):
        
    keys = app.window.keys
    
    if action == keys.ACTION_PRESS:
        
        if key == keys.LEFT:
            state.turn_left = True

        if key == keys.RIGHT:
            state.turn_right = True

        if key == keys.UP:
            state.speed_up = True

        if key == keys.DOWN:
            state.slow_down = True

    if action == keys.ACTION_RELEASE:
        if key == keys.LEFT:
            state.turn_left = False

        if key == keys.RIGHT:
            state.turn_right = False

        if key == keys.UP:
            state.speed_up = False

        if key == keys.DOWN:
            state.slow_down = False

In [17]:
@app.run_me_every(1/120)
def update_pads(ct, dt):
        
    if state.turn_right:
        state.steering_angle = min(3.14159/3.0, state.steering_angle + 0.05)
        
    if state.turn_left:
        state.steering_angle = max(-3.14159/3.0, state.steering_angle - 0.05)
        
    if state.speed_up:
        state.speed = min(50.0, state.speed + 1.0)
        
    if state.slow_down:
        state.speed = max(0.0, state.speed - 1.0)
    
    vx = state.speed * math.cos(state.body_angle)
    vy = state.speed * math.sin(state.body_angle)
    vt = state.speed / state.l * math.tan(state.steering_angle)
    
    state.x += vx * dt
    state.y += vy * dt
    state.body_angle -= vt * dt
    
    body.x = state.x
    body.y = state.y
    body.angle = state.body_angle*180.0/3.14159
    
    r_l = 50.0
    f_l = 50.0
    
    r_wheel.x = body.x - r_l*math.cos(state.body_angle)
    r_wheel.y = body.y - r_l*math.sin(state.body_angle)
    r_wheel.angle = body.angle
    f_wheel.x = body.x + f_l*math.cos(state.body_angle)
    f_wheel.y = body.y + f_l*math.sin(state.body_angle)
    f_wheel.angle = body.angle - state.steering_angle*180/3.14519
    
    body.clip_position(app.width, app.height)
    
    speed.text = str(vt*dt)
    steering_angle.text = str(state.steering_angle*180/3.14159)
    speed.color = 'white'
    steering_angle.color = 'white'
    

In [15]:
@app.run_me
def highlights(ct, dt):
    while True:
        ct, dt = yield 1/24
        speed.color = 'white'
        steering_angle.color = 'white'
        

In [18]:
app.run()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x05\x03\x0…